google eearth engine

In [4]:
import os
from dotenv import load_dotenv,find_dotenv
import ee

# Make sure it loads .env from the correct folder
env_path = find_dotenv()
load_dotenv(dotenv_path=env_path)

SERVICE_GOOGLE = os.getenv("SERVICE_GOOGLE")

if not SERVICE_GOOGLE:
    raise ValueError("❌ SERVICE_GOOGLE not found. Check .env file path or notebook directory.")

print("✅ SERVICE_GOOGLE found:", SERVICE_GOOGLE)

# Initialize Earth Engine
credentials = ee.ServiceAccountCredentials(None, SERVICE_GOOGLE)
ee.Authenticate()
ee.Initialize(credentials)

# Test GEE
image = ee.Image('CGIAR/SRTM90_V4')
stats = image.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=ee.Geometry.Point([10.0, 36.0]),  # Tunisia example
    scale=90
)
print("Mean elevation (Tunisia):", stats.getInfo())


✅ SERVICE_GOOGLE found: sincere-sun-404310-1223ed4b5b60.json


EEException: Cannot authenticate: Invalid request.

In [19]:
tunisia = ee.Geometry.Polygon([
    [[7.5, 30.0], [7.5, 37.5], [11.5, 37.5], [11.5, 30.0]]
])

# 3) Load the ET dataset
et_collection = ee.ImageCollection('FAO/WAPOR/2/L1_AETI_D') \
    .filterDate('2023-01-01', '2024-12-31') \
    .filterBounds(tunisia)

# 4) Compute e.g., total ET over period by summing images
et_sum = et_collection.sum().clip(tunisia)

# Or compute mean daily ET over period
et_mean = et_collection.mean().clip(tunisia)

# 5) Get a reduction for the region (mean value)
mean_dict = et_mean.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=tunisia,
    scale=248.2,   # dataset’s native approximate scale (≈248 m) :contentReference[oaicite:3]{index=3}
    maxPixels=1e10
)

print("Mean ET (mm/day) over Tunisia for 2023-24:", mean_dict.getInfo())

# 6) Optional: export to Google Drive
task = ee.batch.Export.image.toDrive(
    image=et_sum,
    description='ET_Sum_Tunisia_2023_24',
    folder='GEE_exports',
    fileNamePrefix='ET_sum_Tunisia_2023_24',
    region=tunisia,
    scale=248.2,
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()
print("Export started:", task.status())



Mean ET (mm/day) over Tunisia for 2023-24: {'L1_AETI_D': 1.7692290651275793}
Export started: {'state': 'READY', 'description': 'ET_Sum_Tunisia_2023_24', 'priority': 100, 'creation_timestamp_ms': 1761128587431, 'update_timestamp_ms': 1761128587431, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'JT63XHNCFE5LJ3BJEFQXL6HO', 'name': 'projects/earthengine-legacy/operations/JT63XHNCFE5LJ3BJEFQXL6HO'}
